## PIIC - predecir ozono atmosferico
---
**Obj:** predecir el ozono atmosférico $O_3$ máximo del dia siguiente, utilizando la información de químicos de los dias anteriores.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras,gc,time
from keras.layers import *
from keras.models import Sequential,Model

/home/casapanshop/anaconda2/envs/py3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Cargar datos

In [2]:
folder = "../proyecto/datasets/"
df_independencia_verano = pd.read_csv(folder+"dump-Independencia_2018-04-12_230000-verano.csv")
df_independencia_invierno = pd.read_csv(folder+"dump-Independencia_2018-04-12_230000-invierno.csv")

df_condes_verano = pd.read_csv(folder+"dump-Las_Condes_2018-04-12_230000-verano.csv")
df_condes_invierno = pd.read_csv(folder+"dump-Las_Condes_2018-04-12_230000-invierno.csv")

df_independencia_verano.head()

,registered_on,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3
0,1997-11-01 01:00:00,2.7,63.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,1997-11-01 02:00:00,2.6,54.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,1997-11-01 03:00:00,2.2,53.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,1997-11-01 04:00:00,2.5,65.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,1997-11-01 05:00:00,2.3,118.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [3]:
## ultimo año como test
df_condes_verano_train = df_condes_verano[(df_condes_verano["registered_on"]<"2017")]
df_condes_invierno_train = df_condes_invierno[(df_condes_invierno["registered_on"]<"2017")]
df_condes_verano_test = df_condes_verano[df_condes_verano["registered_on"]>="2017"]
df_condes_invierno_test = df_condes_invierno[df_condes_invierno["registered_on"]>="2017"]

df_condes_verano_train.tail()

,registered_on,CH4,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3
69390,2016-12-31 14:00:00,NaN,0.553417,53.0,24.0,7.0400,3.2450,10.28500,NaN,NaN,41.8333,26.6333,NaN,NaN,NaN,NaN,72.5233
69391,2016-12-31 15:00:00,NaN,0.467584,22.0,21.0,4.4825,2.7500,7.23250,NaN,NaN,39.3333,26.3167,NaN,NaN,NaN,NaN,58.9725
69392,2016-12-31 16:00:00,NaN,0.459000,16.5,13.0,3.4925,2.9975,6.49000,NaN,NaN,37.3333,26.0583,NaN,NaN,NaN,NaN,50.3033
69393,2016-12-31 17:00:00,NaN,0.459000,15.0,14.0,3.9050,3.4100,7.31500,NaN,NaN,36.3333,25.1417,NaN,NaN,NaN,NaN,42.9808
69394,2016-12-31 18:00:00,NaN,0.450417,NaN,NaN,6.6275,3.1625,9.79001,NaN,NaN,38.8333,24.1750,NaN,NaN,NaN,NaN,NaN


In [6]:
df_condes_verano_test.head()

,registered_on,CH4,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3
69395,2017-01-03 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,28.834697,NaN
69396,2017-01-03 14:00:00,NaN,NaN,19.5,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,24.715235,NaN
69397,2017-01-03 15:00:00,NaN,0.836667,13.5,18.0,10.2575,1.7600,12.0175,NaN,NaN,14.9167,28.8167,NaN,NaN,0.0,24.537322,73.6176
69398,2017-01-03 16:00:00,NaN,0.768001,19.5,2.0,10.8350,2.5025,13.3375,NaN,NaN,12.2500,28.2917,NaN,NaN,0.0,17.778006,57.6258
69399,2017-01-03 17:00:00,NaN,0.793750,72.0,10.0,11.5775,2.5850,14.1625,NaN,NaN,14.5000,27.4250,NaN,NaN,0.0,9.862870,51.9867


In [7]:
df_condes_verano.dropna(how='any')

,registered_on,CH4,CO,PM10,PM25,NO2,NO,NOX,SO2,WD,RH,TEMP,WS,HCNM,UVA,UVB,O3


Si elimino todas las que tengan al menos un null se eliminan todos los registros

In [8]:
df_condes_verano_train.loc[:,rows_to_use].dropna(subset=["O3"])#(how="all",O3)

NameError: name 'rows_to_use' is not defined

### PRe PRocesarlos

In [57]:
dropnan de o3

In [17]:
## representacion
rows_to_use = ["CO","PM10","PM25","NO2","NO","NOX","SO2","WD","RH","TEMP","WS","HCNM","UVA","UVB","O3"]
from keras.preprocessing.sequence import pad_sequences

def estadisticos(timesteps):
    a_utilizar = [np.nanmax]#,np.nanmin,np.nanstd,np.nanmean]
    return np.concatenate([ function(timesteps,axis=0) for function in a_utilizar])

def create_sequences(dataframe,lag=1,dia=True):
    #preprocesamiento asumiendo dataset es una secuencia
    timestep = dataframe.loc[:,"registered_on"].values
    
    indices_orden = []
    diax = timestep[0].split("-")[-1].split(" ")[0]
    aux_indices1 = 0
    aux_indices2 = 0 
    for dato in timestep[1:]:
        nuevo_dia = dato.split("-")[-1].split(" ")[0]
        aux_indices2+=1
        if diax != nuevo_dia:
            diax = nuevo_dia
            #if aux_indices2-aux_indices1==24:
            #    aux_indices2-=1
            indices_orden.append([aux_indices1,aux_indices2])
            aux_indices1 = aux_indices2
    indices_orden.append([aux_indices1,aux_indices2+1])
    data = dataframe.loc[:,rows_to_use].values
    data = np.asarray([data[init:finit] for init,finit in indices_orden])
    
    horas = 24
    
    dataX = []
    dataY =[]
    for t_plus_1 in range(lag,len(data)):
        #se crea el Y (target)
        predecir = data[t_plus_1][:,-1] #todas las ultimas columnas --sequence
        dataY.append(predecir)
        columnasX = [] #se crea el X (inputs) columnas para predecir Y
        for i in np.arange(lag,0,-1): #para los valores anteriores al t_plus_1 durante un lag
            columnasX.append( data[t_plus_1-i] )
        dataX.append(columnasX)
    
    #mascara delete..
    mask_delete = np.ones(len(dataY),dtype=bool)
    if not dia: 
        for i,dato in enumerate(dataY):
            if np.any(np.isnan(dato)): #uno en sequencia---algun nulo
                mask_delete[i] = False #eliminar de training
    else:
        for i,dato in enumerate(dataY):
            if np.all(np.isnan(dato)): #todos en sequencia
                mask_delete[i] = False #eliminar de training
        
    if dia: #calcular estadisticos
        dataX = [[ estadisticos(timestep) for timestep in datito] for datito in dataX] 
    else:
        aux = [ np.concatenate(datito) for datito in dataX] #junta los lag..
        #pad
        mask = np.full(len(rows_to_use) , -1.)
        dataX = pad_sequences(aux, maxlen=lag*horas, dtype='float32', padding='pre', value=mask) #rellena 
    dataY = pad_sequences(dataY, maxlen=horas, dtype='object', padding='post', value=np.nan) #rellena 
    return np.array(dataX)[mask_delete],np.array(dataY)[mask_delete]

In [18]:
lag = 2
dia = False #a sequencia
trainX1, trainY1 = create_sequences(df_condes_verano_train, lag,dia)
trainX2, trainY2 = create_sequences(df_condes_invierno_train, lag,dia)

testX1, testY1 = create_sequences(df_condes_verano_test, lag,dia)
testX2, testY2 = create_sequences(df_condes_invierno_test, lag,dia)

print("All arrays shapes")
print("Train verano (input): ",trainX1.shape)
print("Train invierno (input): ",trainX2.shape)
print("Train verano (output): ",trainY1.shape)
print("Train invierno (output): ",trainY2.shape)

All arrays shapes
Train verano (input):  (950, 48, 15)
Train invierno (input):  (967, 48, 15)
Train verano (output):  (950, 24)
Train invierno (output):  (967, 24)


In [6]:
#o predecir el maximo en etiqueta...
trainY1 = np.nanmax(trainY1,axis=1)
trainY2 = np.nanmax(trainY2,axis=1)
testY1 =  np.nanmax(testY1,axis=1)
testY2 =  np.nanmax(testY2,axis=1)
print("Maximo calculado")

Maximo calculado


In [10]:
def trid_to_2d(array,backward=False):
    if backward:
        return array.reshape(array.shape[0]/lag, lag ,array.shape[-1])
    else:
        return array.reshape((np.prod(array.shape[:2]), array.shape[-1]))

In [11]:
trid_to_2d(trid_to_2d(trainX),backward=True).shape

NameError: name 'trainX' is not defined

### Escalar datos y transformar para entrada

In [19]:
#calculo de estadisticos ("fit") MinMaxScaler
class MinMax(object):
    def __init__(self,aux=True):
        self.minimos = None
        self.maximos = None
        
    def fit(self,data):
        self.minimos = np.nanmin(data, axis=(0,1))#, keepdims=True)
        self.maximos = np.nanmax(data ,axis=(0,1))#, keepdims=True)
    
    def transform(self,data):
        #normalize between 0 and 1 every statisctic
        return (data-self.minimos) / (self.maximos-self.minimos)
    
    def inverse_transform(self,data):
        return data*(self.maximos-self.minimos) + self.minimos
    
scaler_model1 = MinMax()
scaler_model1.fit(trainX1)
X_train1 = scaler_model1.transform(trainX1)
X_test1 = scaler_model1.transform(testX1)

scaler_model2 = MinMax()
scaler_model2.fit(trainX2)
X_train2 = scaler_model2.transform(trainX2)
X_test2 = scaler_model2.transform(testX2)

X_train1.shape

(950, 48, 15)

In [20]:
print("Minimos en train: ", np.nanmin(X_train1, axis=(0,1)))
print("Minimos en test: ", np.nanmin(X_test1, axis=(0,1)))

Minimos en train:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Minimos en test:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [21]:
print("Maximos en train: ", np.nanmax(X_train2, axis=(0,1)))
print("Maximos en test: ", np.nanmax(X_test2, axis=(0,1)))

Maximos en train:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Maximos en test:  [0.34242496 0.3527508  0.52879584 0.67041296 0.83688116 0.8192184
 0.         0.         0.9509843  0.8743269  0.         0.
 0.04304834 0.26231417 0.37820512]


In [22]:
mask_value = -1.
def nan_to_num(data,mask_value):
    return np.asarray([[[mask_value if np.isnan(x) else x for x in timestep] for timestep in array] for array in data])

X_train1 = nan_to_num(X_train1,mask_value)
X_test1 = nan_to_num(X_test1,mask_value)
X_train2 = nan_to_num(X_train2,mask_value)
X_test2 = nan_to_num(X_test2,mask_value)
print("Mascara agregada")

Mascara agregada


In [23]:
def nan_to_num(data,mask_value):
    return np.asarray([[mask_value if np.isnan(x) else x for x in array] for array in data])

y_train1 = nan_to_num(trainY1,mask_value)
y_test1 = nan_to_num(testY1,mask_value)
y_train2 = nan_to_num(trainY2,mask_value)
y_test2 = nan_to_num(testY2,mask_value)
print("Mascara agregada en etiqueta")

Mascara agregada en etiqueta


#para el Y tambien...
from sklearn.preprocessing import MinMaxScaler
#o standar
from sklearn.preprocessing import StandardScaler

#scaler_model = MinMaxScaler()
scaler_model = StandardScaler()
scaler_model.fit(trainY1)
y_train1 = scaler_model.transform(trainY1)
y_test1 = scaler_model.transform(testY1)
y_train1.shape

Características del modelo:
* Predecir el máximo de ozono $O_3$ del día siguiente (del período de 24 horas).
* Grano de predicción: Horas o día?
* Cuanta información anterior utilizar? ..3 dias
* Nulos? en $O_3$ o contaminantes?
    * Borrar
    * Mascara

## Modelo

In [11]:
batch_size = 32
epochs = 20
opt = 'adam'
loss_utilizada = 'mse' #?
nhidden = 64
from attention import *

In [12]:
#https://github.com/farizrahman4u/seq2seq
import seq2seq
from seq2seq.models import Seq2Seq
from seq2seq.models import AttentionSeq2Seq


model = Seq2Seq(input_dim=5, hidden_dim=10, output_length=8, output_dim=8)
model.compile(loss='mse', optimizer='rmsprop')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, None, 5)      0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, None, 10)     60          input_7[0][0]                    
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
private__optional_input_place_h (2,)                 0                                            
__________________________________________________________________________________________________
private__o

In [36]:
#from seq2seq.models import Seq2Seq,AttentionSeq2Seq

timesteps = X_train1.shape[1]
features = X_train1.shape[2] #todo excepto o3

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
def attention_multiply(vects):
    encoder, attention = vects
    return K.batch_dot(attention,encoder, axes=1)

def modelo(tipo,dia=True):
    regularization = 0.005
    model = Sequential()
    model.add(Masking(mask_value=mask_value, input_shape=(timesteps, features)))

    if tipo=="sin atencion":
        model.add(GRU(nhidden,return_sequences=True,kernel_regularizer=keras.regularizers.l2(regularization)))
        model.add(GRU(nhidden,return_sequences=True,kernel_regularizer=keras.regularizers.l2(regularization)))
    elif tipo=="atencion":
        model.add(Bidirectional(GRU(nhidden, return_sequences=True,kernel_regularizer=keras.regularizers.l2(regularization))))
        model.add(AttentionDecoder(nhidden, nhidden,kernel_regularizer=keras.regularizers.l2(regularization))) #que son esos  (units, output_dim)
    
    elif tipo=="sin atencion libreria":
        aux_model = Seq2Seq(teacher_force=True,input_dim=features, hidden_dim=nhidden, output_length=24, output_dim=1,depth=3)
        #teacherforse=True
        #aux_model = Model(inputs=model.inputs,outputs=aux_model(model.outputs))
        
        #layer_aux =  GRU(nhidden, return_sequences=False)(aux_model.outputs)
        #layer_aux = Dense(1,activation='linear')(layer_aux) #1 prediccion?
        
        #model = Model(inputs=aux_model.inputs,outputs=layer_aux)
        aux_model.compile(loss=loss_utilizada,optimizer=opt)
        aux_model.summary()
        return aux_model
    
    elif tipo=="atencion libreria":
        aux_model = AttentionSeq2Seq(input_dim=features,input_length=timesteps,hidden_dim=nhidden
                                     ,output_length=timesteps,output_dim=nhidden,depth=2)
        model = Sequential()
        model.add(aux_model)
    
    if dia:
        model.add(GRU(nhidden, return_sequences=False,kernel_regularizer=keras.regularizers.l2(regularization)))
        model.add(Dropout(0.3))
        model.add(Dense(1,activation='linear',kernel_regularizer=keras.regularizers.l2(regularization))) #1 prediccion?
    else:
        model.add(TimeDistributed(Dense(1,activation='linear',kernel_regularizer=keras.regularizers.l2(regularization)))) #varias prediccions--a nivel de horario..
        #model.add(GlobalMaxPooling1D()) #no soporta maskin
        model.add(GRU(1,return_sequences=False))
    
    model.compile(loss=loss_utilizada,optimizer=opt)
    model.summary()
    return model

modelo1 = modelo("sin atencion",dia)
modelo2 = modelo("atencion",dia)
modelo3 = modelo("sin atencion libreria",dia)
modelo4 = modelo("atencion libreria",dia)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_13 (Masking)         (None, 48, 15)            0         
_________________________________________________________________
gru_19 (GRU)                 (None, 48, 64)            15360     
_________________________________________________________________
gru_20 (GRU)                 (None, 48, 64)            24768     
_________________________________________________________________
time_distributed_14 (TimeDis (None, 48, 1)             65        
_________________________________________________________________
gru_21 (GRU)                 (None, 1)                 9         
Total params: 40,202
Trainable params: 40,202
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
maskin

In [109]:
modelo1.fit(X_train1, trainY1, epochs=epochs, batch_size=batch_size, validation_data=(X_test1, testY1))

Train on 2931 samples, validate on 195 samples
Epoch 1/20
2931/2931 [==============================] - 13s 4ms/step - loss: 5583.4554 - val_loss: 3220.1545
Epoch 2/20
2931/2931 [==============================] - 1s 315us/step - loss: 4336.0182 - val_loss: 2608.2805
Epoch 3/20
2931/2931 [==============================] - 1s 329us/step - loss: 3654.8928 - val_loss: 2122.1171
Epoch 4/20
2931/2931 [==============================] - 1s 333us/step - loss: 3083.6249 - val_loss: 1725.8915
Epoch 5/20
2931/2931 [==============================] - 1s 326us/step - loss: 2606.8954 - val_loss: 1404.9029
Epoch 6/20
2931/2931 [==============================] - 1s 309us/step - loss: 2201.6552 - val_loss: 1147.0733
Epoch 7/20
2931/2931 [==============================] - 1s 306us/step - loss: 1884.7589 - val_loss: 946.0074
Epoch 8/20
2931/2931 [==============================] - 1s 304us/step - loss: 1608.3693 - val_loss: 791.6010
Epoch 9/20
2931/2931 [==============================] - 1s 314us/step - loss

In [14]:
modelo2.fit(X_train1, trainY1, epochs=epochs, batch_size=batch_size, validation_data=(X_test1, testY1))

Train on 950 samples, validate on 43 samples
Epoch 1/20
950/950 [==============================] - 20s 21ms/step - loss: 8406.8187 - val_loss: 6237.8115
Epoch 2/20
950/950 [==============================] - 12s 13ms/step - loss: 8377.3369 - val_loss: 6223.3930
Epoch 3/20
950/950 [==============================] - 12s 13ms/step - loss: 8361.1848 - val_loss: 6209.8784
Epoch 4/20
950/950 [==============================] - 12s 13ms/step - loss: 8346.2092 - val_loss: 6197.5575
Epoch 5/20
950/950 [==============================] - 12s 13ms/step - loss: 8332.6855 - val_loss: 6186.6649
Epoch 6/20
950/950 [==============================] - 12s 13ms/step - loss: 8320.8717 - val_loss: 6177.2271
Epoch 7/20
950/950 [==============================] - 12s 13ms/step - loss: 8310.7392 - val_loss: 6169.2814
Epoch 8/20
950/950 [==============================] - 12s 13ms/step - loss: 8302.2670 - val_loss: 6162.6480
Epoch 9/20
950/950 [==============================] - 12s 13ms/step - loss: 8295.2572 - val

In [31]:
aux_trainy = y_train1.reshape(y_train1.shape[0],y_train1.shape[1],1)

In [37]:
#modelo3.fit(X_train1, trainY1, epochs=epochs, batch_size=batch_size, validation_data=(X_test1, testY1))
modelo3.fit([X_train1,aux_trainy], aux_trainy, epochs=epochs, batch_size=batch_size)# validation_split=0.2)#validation_data=(X_test1, testY1))

Epoch 1/20
950/950 [==============================] - 24s 25ms/step - loss: 1604.5005
Epoch 2/20
950/950 [==============================] - 13s 13ms/step - loss: 1585.3703
Epoch 3/20
950/950 [==============================] - 13s 13ms/step - loss: 1585.3064
Epoch 4/20
950/950 [==============================] - 13s 13ms/step - loss: 1585.3000
Epoch 5/20
950/950 [==============================] - 13s 13ms/step - loss: 1585.2947
Epoch 6/20
950/950 [==============================] - 13s 13ms/step - loss: 1585.2899
Epoch 7/20
950/950 [==============================] - 13s 13ms/step - loss: 1585.2855
Epoch 8/20
950/950 [==============================] - 13s 13ms/step - loss: 1585.2810
Epoch 9/20
950/950 [==============================] - 13s 13ms/step - loss: 1585.2768
Epoch 10/20
950/950 [==============================] - 13s 13ms/step - loss: 1585.2727
Epoch 11/20
950/950 [==============================] - 13s 13ms/step - loss: 1585.2686
Epoch 12/20
950/950 [==============================]

In [ ]:
probar= modelo3.predict(X_train1)
probar

In [53]:
modelo4.fit(X_train1, trainY1, epochs=epochs, batch_size=batch_size, validation_data=(X_test1, testY1))

Train on 2931 samples, validate on 195 samples
Epoch 1/15
2931/2931 [==============================] - 13s 4ms/step - loss: 5641.3066 - val_loss: 3279.1513
Epoch 2/15
2931/2931 [==============================] - 3s 1ms/step - loss: 4395.8773 - val_loss: 2658.1465
Epoch 3/15
2931/2931 [==============================] - 3s 1ms/step - loss: 3699.4842 - val_loss: 2159.9836
Epoch 4/15
2931/2931 [==============================] - 3s 1ms/step - loss: 3119.7706 - val_loss: 1755.7045
Epoch 5/15
2931/2931 [==============================] - 3s 1ms/step - loss: 2632.7764 - val_loss: 1427.1046
Epoch 6/15
2931/2931 [==============================] - 3s 1ms/step - loss: 2224.0965 - val_loss: 1164.2534
Epoch 7/15
2931/2931 [==============================] - 3s 1ms/step - loss: 1883.1651 - val_loss: 957.8679
Epoch 8/15
2931/2931 [==============================] - 3s 1ms/step - loss: 1602.1872 - val_loss: 798.1491
Epoch 9/15
2931/2931 [==============================] - 3s 1ms/step - loss: 1372.0597 - va

In [ ]:
seguir con modelo 3 y modelo 4

#ivnierno
modelo1.fit(X_train2, trainY2, epochs=epochs, batch_size=batch_size, validation_data=(X_test2, testY2))
modelo2.fit(X_train2, trainY2, epochs=epochs, batch_size=batch_size, validation_data=(X_test2, testY2))

In [117]:
## evaluación?
rmse?

from sklearn.metrics import mean_squared_error
def rmse_evaluation(model,input_data,output_data):
    predict = model.predict(input_data)
    #recover data
    predict = scaler_model.inverse_transform(predict)
    output_data = scaler_model.inverse_transform(output_data)
    print("RMSE del modelo en test: ",np.sqrt(mean_squared_error(output_data,predict)))
    return 

def evaluar_modelo(modelito,data):
    mse_test = modelito.evaluate(data[0],data[1])
    print("RMSE del modelo en test: ",np.sqrt(mse_test))
    return np.sqrt(mse_test)
#if es hora.. max
# si es por dia ya predice max

Object `rmse` not found.


In [111]:
evaluar_modelo(modelo1,[X_test1,testY1])   

195/195 [==============================] - 0s 291us/step
RMSE del modelo en test:  23.506534082171495


In [118]:
rmse_evaluation(modelo2,X_test1,y_test1)  

RMSE del modelo en test:  25.499195690214826


In [119]:
evaluar_modelo(modelo2,[X_test1,testY1])      

195/195 [==============================] - 0s 390us/step
RMSE del modelo en test:  72.43425606591757


In [ ]:
#entrenar 1 modelo por cada conjunto y promediar o entrenar juntos

In [138]:
## mejor modelo atencion normal
modelo_v = modelo("atencion",dia)
modelo_i = modelo("atencion",dia)
modelo_v.fit(X_train1, trainY1, epochs=epochs, batch_size=batch_size, validation_data=(X_test1, testY1))
modelo_i.fit(X_train2, trainY2, epochs=epochs, batch_size=batch_size, validation_data=(X_test2, testY2))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_67 (Masking)         (None, 3, 15)             0         
_________________________________________________________________
bidirectional_28 (Bidirectio (None, 3, 128)            30720     
_________________________________________________________________
AttentionDecoder (AttentionD (None, 3, 64)             86400     
_________________________________________________________________
gru_161 (GRU)                (None, 64)                24768     
_________________________________________________________________
dropout_79 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 65        
Total params: 141,953
Trainable params: 141,953
Non-trainable params: 0
_________________________________________________________________
____

In [139]:
rmse_1 = evaluar_modelo(modelo_v,[X_test1,testY1])    
rmse_2 = evaluar_modelo(modelo_i,[X_test2,testY2])    
print("En promedio este modelo da: ",np.mean([rmse_1,rmse_2]))

195/195 [==============================] - 0s 371us/step
RMSE del modelo en test:  23.991938999518787
116/116 [==============================] - 0s 340us/step
RMSE del modelo en test:  20.900546881842484
En promedio este modelo da:  22.446242940680634


In [140]:
## evaluación?
from sklearn.metrics import mean_squared_error
predict1 = modelo_v.predict(X_test1)
predict2 = modelo_i.predict(X_test2)

print("RMSE del modelo en test: ",np.sqrt(mean_squared_error(y_new_test,np.concatenate((predict1,predict2)))))

RMSE del modelo en test:  22.86701676641055


In [266]:
#stack invierno y verano
X_new_train = np.concatenate((X_train1,X_train2))
X_new_test = np.concatenate((X_test1,X_test2))
y_new_train = np.concatenate((trainY1,trainY2))
y_new_test = np.concatenate((testY1,testY2))
X_new_train.shape

(2004, 69, 15)

In [144]:
modelo2_both = modelo("atencion",dia)
modelo2_both.fit(X_new_train, y_new_train, epochs=25, batch_size=batch_size, validation_data=(X_new_test, y_new_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_70 (Masking)         (None, 3, 15)             0         
_________________________________________________________________
bidirectional_31 (Bidirectio (None, 3, 128)            30720     
_________________________________________________________________
AttentionDecoder (AttentionD (None, 3, 64)             86400     
_________________________________________________________________
gru_167 (GRU)                (None, 64)                24768     
_________________________________________________________________
dropout_82 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_69 (Dense)             (None, 1)                 65        
Total params: 141,953
Trainable params: 141,953
Non-trainable params: 0
_________________________________________________________________
Trai

In [145]:
evaluar_modelo(modelo2_both,[X_new_test,y_new_test])    

311/311 [==============================] - 0s 344us/step
RMSE del modelo en test:  17.199719013374132


17.199719013374132

### Resultados: Entrenar un mismo modelo en invierno y verano es mejor..
### Conclusiones: Los patrones relacionados con la tarea asignada se repiten/son compartidos en ambos conjuntos

## Mejorar
---
* seguir con atención
* Probar predecir secuencia completa (horas)---otro archivo
* juntar con independencia???
* aumentar lag?

### Independencia